In [ ]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoTokenizer, AutoConfig
import pandas as pd
import torch
import shutil # Pour nettoyer si besoin

print("⏳ Chargement des modèles... (Méthode infaillible)")

# --- 1. SNOWFLAKE (Le modèle complexe) ---
# On va le "réparer" manuellement avant de le donner à SentenceTransformer
model_id_snow = 'Snowflake/snowflake-arctic-embed-m-v2.0'

# A. On configure manuellement pour éviter l'erreur xformers
config = AutoConfig.from_pretrained(model_id_snow, trust_remote_code=True)
config.use_memory_efficient_attention = False  # Le FIX critique
config.unpad_inputs = False

# B. On charge avec Hugging Face standard (ça, ça marche toujours)
print("   ... Téléchargement et correction de Snowflake")
hf_model = AutoModel.from_pretrained(model_id_snow, config=config, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id_snow)

# C. On sauvegarde le modèle corrigé dans un dossier local
save_path = "./snowflake_fixed_local"
hf_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# D. On charge SentenceTransformer depuis ce dossier local
# Maintenant il lit la config locale qui contient déjà notre correctif !
model_snowflake = SentenceTransformer(save_path, trust_remote_code=True)


# --- 2. SOLON (Le modèle standard) ---
# Pour lui, la méthode simple suffit généralement.
# Si ça plante, on utilisera une méthode similaire, mais testons simple d'abord.
print("   ... Chargement de Solon")
model_solon = SentenceTransformer('OrdalieTech/SOLON-embeddings-large-0.1')


print("✅ TOUS LES MODÈLES SONT CHARGÉS !")

# --- 3. LE DUEL DE JARGON ---
test_cases = [
    {
        "terme": "PLF",
        "candidats": [
            "Projet de Loi de Finances présenté au parlement", # Cible
            "Playlist Favorites de musique",                   # Piège
            "Pour La France rassemblement politique"           # Hallucination
        ]
    },
    {
        "terme": "Liquidation",
        "candidats": [
            "Procédure judiciaire mettant fin à une société commerciale", # Cible
            "Vente totale de marchandises à prix réduits en magasin",     # Piège
            "Action de rendre liquide une substance gazeuse"              # Piège
        ]
    },
    {
        "terme": "Assiette",
        "candidats": [
            "Base de calcul sur laquelle est appliqué un taux d'imposition", # Cible
            "Pièce de vaisselle plate pour servir les repas",                # Piège
            "Position d'équilibre d'un avion en vol"                         # Piège
        ]
    }
]

def comparer_definitions(model, model_name, tests):
    resultats = []
    for t in tests:
        anchor = t["terme"]
        candidates = t["candidats"]

        # Encodage
        emb_anchor = model.encode(anchor, convert_to_tensor=True)
        emb_candidates = model.encode(candidates, convert_to_tensor=True)

        # Similarité
        scores = util.cos_sim(emb_anchor, emb_candidates)[0]
        best_score_idx = torch.argmax(scores).item()
        best_score = scores[best_score_idx].item()
        choix = candidates[best_score_idx]

        resultats.append({
            "Modèle": model_name,
            "Terme": anchor,
            "Choix du Modèle": choix,
            "Score": f"{best_score:.4f}",
            "Validé ?": "✅" if best_score_idx == 0 else "❌"
        })
    return resultats

# Exécution
res_snowflake = comparer_definitions(model_snowflake, "Snowflake", test_cases)
res_solon = comparer_definitions(model_solon, "Solon", test_cases)

# Affichage
df = pd.DataFrame(res_snowflake + res_solon)
df = df.sort_values(by="Terme")
print("\n🏆 RÉSULTATS FINAUX 🏆")
print(df[["Terme", "Modèle", "Choix du Modèle", "Score", "Validé ?"]].to_markdown(index=False))

⏳ Chargement des modèles... (Méthode infaillible)
   ... Téléchargement et correction de Snowflake


modeling_hf_alibaba_nlp_gte.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Snowflake/snowflake-arctic-embed-m-v2.0:
- modeling_hf_alibaba_nlp_gte.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

✅ TOUS LES MODÈLES SONT CHARGÉS !

🏆 RÉSULTATS FINAUX 🏆
| Terme       | Modèle    | Choix du Modèle                                            |   Score | Validé ?   |
|:------------|:----------|:-----------------------------------------------------------|--------:|:-----------|
| Assiette    | Snowflake | Pièce de vaisselle plate pour servir les repas             |  0.4882 | ❌         |
| Assiette    | Solon     | Pièce de vaisselle plate pour servir les repas             |  0.5197 | ❌         |
| Liquidation | Snowflake | Procédure judiciaire mettant fin à une société commerciale |  0.5561 | ✅         |
| Liquidation | Solon     | Procédure judiciaire mettant fin à une société commerciale |  0.3531 | ✅         |
| PLF         | Snowflake | Projet de Loi de Finances présenté au parlement            |  0.2917 | ✅         |
| PLF         | Solon     | Pour La France rassemblement politique                     |  0.2854 | ❌         |


In [ ]:
import pandas as pd
import torch
from sentence_transformers import util

# 1. LE DATASET "MINI-BERCY" (20 Termes Réalistes)
# Chaque entrée est un couple (Terme, Définition Correcte)
dataset_bercy = [
    ("TVA", "Impôt indirect sur la consommation payé par le consommateur final."),
    ("IR (Impôt sur le Revenu)", "Impôt direct calculé au niveau du foyer fiscal portant sur l'ensemble des revenus."),
    ("IS (Impôt sur les Sociétés)", "Taxe prélevée sur le résultat annuel des entreprises exploitées en France."),
    ("Prélèvement à la source", "Mode de recouvrement de l'impôt consistant à le faire payer au moment du versement du revenu."),
    ("Assiette fiscale", "Montant servant de base au calcul d'un impôt ou d'une taxe."),
    ("Recouvrement", "Ensemble des procédures pour obtenir le paiement d'une somme due au Trésor public."),
    ("Rescrit fiscal", "Réponse de l'administration à une question d'un contribuable sur sa situation."),
    ("Niche fiscale", "Avantage fiscal permettant de diminuer le montant de l'impôt dû."),
    ("Dégrèvement", "Suppression ou atténuation partielle de l'impôt accordée par l'administration."),
    ("Redressement fiscal", "Opération de rectification par l'administration d'une imposition insuffisante."),
    ("Amortissement comptable", "Constatation comptable de la perte de valeur d'un actif du fait de l'usure ou du temps."),
    ("Bilan comptable", "Document synthétisant le patrimoine de l'entreprise à un instant T (Actif / Passif)."),
    ("Compte de résultat", "Document comptable présentant les produits et les charges d'un exercice."),
    ("Déficit budgétaire", "Situation où les dépenses de l'État sont supérieures à ses recettes."),
    ("Dette publique", "Ensemble des engagements financiers pris par l'État et les administrations publiques."),
    ("Loi de Finances (PLF)", "Acte législatif prévoyant et autorisant le budget de l'État pour l'année à venir."),
    ("Marché public", "Contrat conclu à titre onéreux par un acheteur public pour répondre à ses besoins."),
    ("Cadastre", "Registre public définissant l'emplacement et la valeur des propriétés foncières."),
    ("Droits de succession", "Impôt prélevé sur la transmission du patrimoine d'une personne décédée."),
    ("Plus-value", "Gain réalisé lors de la vente d'un bien à un prix supérieur à son prix d'achat.")
]

# On sépare les termes et les définitions
termes = [item[0] for item in dataset_bercy]
definitions = [item[1] for item in dataset_bercy]

def evaluer_modele_global(model, model_name):
    print(f"--- Évaluation de {model_name} ---")

    # 1. On encode TOUT (Termes et Définitions)
    # C'est rapide : on vectorise en lot
    emb_termes = model.encode(termes, convert_to_tensor=True)
    emb_defs = model.encode(definitions, convert_to_tensor=True)

    # 2. Matrice de similarité (20x20)
    # Pour chaque terme, on regarde sa similarité avec TOUTES les définitions
    # scores_matrix[i][j] = score entre le terme i et la définition j
    scores_matrix = util.cos_sim(emb_termes, emb_defs)

    top1_accuracy = 0
    top3_accuracy = 0
    erreurs = []

    # 3. Analyse des résultats
    for i, terme_reel in enumerate(termes):
        scores_du_terme = scores_matrix[i] # Ligne i

        # On cherche les index des meilleures définitions
        # topk renvoie (valeurs, indices)
        top_results = torch.topk(scores_du_terme, k=3)
        top_indices = top_results.indices.tolist()

        # Est-ce que la bonne définition (index i) est dans le Top 1 ?
        if top_indices[0] == i:
            top1_accuracy += 1
        else:
            # Si erreur, on note ce qu'il a confondu pour l'analyse
            faux_choix_index = top_indices[0]
            erreurs.append({
                "Terme": terme_reel,
                "A choisi (Erreur)": termes[faux_choix_index], # Quel terme correspond à la def qu'il a choisie ?
                "Score Faux": f"{scores_du_terme[faux_choix_index]:.3f}",
                "Score Vrai": f"{scores_du_terme[i]:.3f}"
            })

        # Est-ce que la bonne définition est dans le Top 3 ?
        if i in top_indices:
            top3_accuracy += 1

    # Calcul des pourcentages
    score_final_top1 = (top1_accuracy / len(termes)) * 100
    score_final_top3 = (top3_accuracy / len(termes)) * 100

    return {
        "name": model_name,
        "Top-1 Accuracy": score_final_top1,
        "Top-3 Accuracy": score_final_top3,
        "erreurs": erreurs
    }

# --- EXÉCUTION DU TEST ---
# (On suppose que model_snowflake et model_solon sont déjà chargés grâce au code précédent)

resultats_snow = evaluer_modele_global(model_snowflake, "Snowflake (Arctic)")
resultats_solon = evaluer_modele_global(model_solon, "Solon (Juridique)")

# --- AFFICHAGE DU SCOREBOARD ---
summary = pd.DataFrame([
    {"Modèle": res["name"], "Précision Top-1 (%)": res["Top-1 Accuracy"], "Précision Top-3 (%)": res["Top-3 Accuracy"]}
    for res in [resultats_snow, resultats_solon]
])

print("\n📊 RÉSULTATS DU BENCHMARK 'BERCY-20' 📊")
print(summary.to_markdown(index=False))

# --- ANALYSE DES ERREURS (Pour comprendre POURQUOI) ---
print("\n🕵️ ANALYSE DES CONFUSIONS (Top 3 erreurs par modèle)")

for res in [resultats_snow, resultats_solon]:
    print(f"\nModèle : {res['name']}")
    if not res["erreurs"]:
        print("   ✅ Aucune erreur ! Carton plein.")
    else:
        for err in res["erreurs"][:3]: # On affiche les 3 premières erreurs
            print(f"   ⚠️  Pour '{err['Terme']}', il a préféré la déf de '{err['A choisi (Erreur)']}' (Score: {err['Score Faux']} vs Vrai: {err['Score Vrai']})")

--- Évaluation de Snowflake (Arctic) ---
--- Évaluation de Solon (Juridique) ---

📊 RÉSULTATS DU BENCHMARK 'BERCY-20' 📊
| Modèle             |   Précision Top-1 (%) |   Précision Top-3 (%) |
|:-------------------|----------------------:|----------------------:|
| Snowflake (Arctic) |                    60 |                    80 |
| Solon (Juridique)  |                    65 |                    80 |

🕵️ ANALYSE DES CONFUSIONS (Top 3 erreurs par modèle)

Modèle : Snowflake (Arctic)
   ⚠️  Pour 'TVA', il a préféré la déf de 'Assiette fiscale' (Score: 0.355 vs Vrai: 0.316)
   ⚠️  Pour 'Prélèvement à la source', il a préféré la déf de 'IS (Impôt sur les Sociétés)' (Score: 0.466 vs Vrai: 0.438)
   ⚠️  Pour 'Assiette fiscale', il a préféré la déf de 'Niche fiscale' (Score: 0.418 vs Vrai: 0.397)

Modèle : Solon (Juridique)
   ⚠️  Pour 'TVA', il a préféré la déf de 'Assiette fiscale' (Score: 0.206 vs Vrai: 0.144)
   ⚠️  Pour 'Assiette fiscale', il a préféré la déf de 'IR (Impôt sur le Revenu)

pré-requis

In [ ]:
# Installation des librairies nécessaires sur Colab
!pip install -q sentence-transformers pandas

import pandas as pd
import torch
import os
import shutil
from sentence_transformers import SentenceTransformer, util, InputExample, losses, evaluation, models
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer, AutoConfig


print("⏳ Chargement des modèles en cours...")

# --- 1. CONFIGURATION ET CHARGEMENT DE SNOWFLAKE (Le modèle complexe) ---
model_id_snow = 'Snowflake/snowflake-arctic-embed-m-v2.0'
path_snow_fixed = "./snowflake_fixed_local"

# On nettoie le dossier s'il existe déjà pour être sûr de repartir propre
if os.path.exists(path_snow_fixed):
    shutil.rmtree(path_snow_fixed)

print("   ... Téléchargement et correction de la config Snowflake")
try:
    # On force la désactivation de l'option qui fait planter Colab
    config = AutoConfig.from_pretrained(model_id_snow, trust_remote_code=True)
    config.use_memory_efficient_attention = False
    config.unpad_inputs = False

    # On télécharge avec la config corrigée
    hf_model = AutoModel.from_pretrained(model_id_snow, config=config, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id_snow)

    # On sauvegarde en local
    hf_model.save_pretrained(path_snow_fixed)
    tokenizer.save_pretrained(path_snow_fixed)

    # On charge SentenceTransformer depuis le dossier local corrigé
    model_snowflake = SentenceTransformer(path_snow_fixed, trust_remote_code=True)
    print("   ✅ Snowflake chargé avec succès.")
except Exception as e:
    print(f"   ❌ Erreur Snowflake : {e}")

# --- 2. CHARGEMENT DE SOLON (Le modèle souverain) ---
try:
    print("   ... Chargement de Solon")
    # Méthode explicite (Transformer + Pooling) pour compatibilité maximale
    word_embedding_model = models.Transformer('OrdalieTech/SOLON-embeddings-large-0.1')
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode='mean')
    model_solon = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    print("   ✅ Solon chargé avec succès.")
except Exception as e:
    print(f"   ❌ Erreur Solon : {e}")

print("\n🚀 Prêt pour les tests !")

⏳ Chargement des modèles en cours...
   ... Téléchargement et correction de la config Snowflake


   ✅ Snowflake chargé avec succès.
   ... Chargement de Solon
   ✅ Solon chargé avec succès.

🚀 Prêt pour les tests !


Méthode 1

In [ ]:
print("🥊 LANCEMENT MÉTHODE 1 : TEST DE JARGON (60 TERMES)")

# Dataset : Terme / Sens Bercy (Bon) / Sens Courant ou Piège (Mauvais)
dataset_jargon = [
    # FISCALITÉ
    ("Assiette", "Base de calcul de l'impôt", "Vaisselle pour manger"),
    ("Liquidation", "Calcul du montant de l'impôt exigible", "Soldes en magasin"),
    ("Recouvrement", "Action de percevoir l'impôt", "Couvrir un objet"),
    ("Role", "Titre exécutoire collectif pour l'impôt", "Personnage de cinéma"),
    ("Foyer", "Unité d'imposition (Foyer fiscal)", "Cheminée ou lieu d'habitation"),
    ("Abattement", "Déduction sur la base imposable", "Découragement ou fatigue"),
    ("Grace", "Remise gracieuse d'une dette fiscale", "Élégance ou prière"),
    ("Timbre", "Taxe payée pour une formalité (Timbre fiscal)", "Vignette postale de collection"),
    ("Retenue", "Somme prélevée à la source", "Punition scolaire"),
    ("Imputation", "Application d'un crédit d'impôt sur une somme due", "Accusation d'une faute"),
    ("Centimes", "Centimes additionnels (taxes locales)", "Petites pièces de monnaie"),
    ("Contribution", "Impôt spécifique (CSG, CRDS)", "Apport d'idées à un projet"),
    ("Exploitant", "Personne physique exerçant une activité agricole ou commerciale", "Personne qui profite des autres"),
    ("Redevance", "Somme versée en contrepartie d'un service public (Audiovisuel)", "Loyer féodal historique"),
    ("Vacance", "État d'un logement vide (Taxe sur les logements vacants)", "Période de congés scolaires"),

    # COMPTABILITÉ PUBLIQUE
    ("Exercice", "Période budgétaire annuelle", "Entraînement sportif"),
    ("Ventilation", "Répartition budgétaire par mission", "Système d'aération"),
    ("Engagement", "Acte juridique créant une dette pour l'État", "Promesse de mariage"),
    ("Liquide", "Disponibilités financières immédiates", "État de l'eau"),
    ("Balance", "État récapitulatif des comptes", "Instrument de pesée"),
    ("Provision", "Réserve comptable pour risque", "Nourriture pour voyage"),
    ("Ecriture", "Enregistrement d'une opération comptable", "Calligraphie ou style littéraire"),
    ("Mandat", "Ordre de payer une dépense publique", "Durée de fonction d'un élu"),
    ("Titre", "Justificatif de recette (Titre de perception)", "Nom d'un livre"),
    ("Chapitre", "Subdivision d'un budget", "Partie d'un roman"),
    ("Article", "Plus petite subdivision d'exécution budgétaire", "Texte de journal"),
    ("Amortissement", "Constatation de la perte de valeur d'un bien", "Réduction des chocs"),
    ("Liasse", "Ensemble des documents fiscaux (Liasse fiscale)", "Paquet de billets"),
    ("Clôture", "Arrêt des comptes en fin d'année", "Barrière de jardin"),
    ("Immobilisation", "Bien durable détenu par l'entité", "Plâtre médical sur une jambe"),

    # JURIDIQUE & ADMIN
    ("Instruction", "Texte précisant l'application d'une loi (Bofip)", "Éducation des enfants"),
    ("Arrêté", "Décision administrative d'une autorité", "Fait d'être stoppé"),
    ("Disposition", "Article d'un texte de loi", "Arrangement de meubles"),
    ("Mutation", "Changement de propriétaire d'un bien (Droits de mutation)", "Transformation biologique"),
    ("Minute", "Original d'un jugement ou acte notarié", "Unité de temps de 60 secondes"),
    ("Grosse", "Copie exécutoire d'un jugement", "Personne en surpoids (vulgaire)"),
    ("Saisie", "Confiscation de biens par l'huissier", "Frappe au clavier"),
    ("Parquet", "Ministère public (Magistrats)", "Sol en bois"),
    ("Siège", "Lieu où est établie la direction d'une personne morale", "Chaise ou fauteuil"),
    ("Ressort", "Étendue de la compétence d'une juridiction", "Pièce mécanique élastique"),
    ("Frais", "Dépenses professionnelles déductibles", "Température froide"),
    ("Service", "Division administrative (Service des impôts)", "Mise de table"),
    ("Corps", "Ensemble de fonctionnaires (Grand corps de l'État)", "Partie physique de l'homme"),
    ("Cadre", "Catégorie d'emploi (Cadre A, B, C)", "Bordure de tableau"),
    ("Mouvement", "Mutation ou affectation de fonctionnaires", "Déplacement physique"),

    # FINANCE & ÉCONOMIE
    ("Obligation", "Titre de dette émis par l'État (OAT)", "Devoir moral"),
    ("Action", "Part du capital d'une entreprise", "Film d'aventure"),
    ("Trésor", "L'État banquier et caissier", "Coffre de pirate"),
    ("Bulle", "Surévaluation des actifs", "Sphère de savon"),
    ("Plafond", "Limite maximale (Plafond de la sécu)", "Haut d'une pièce"),
    ("Plancher", "Limite minimale", "Sol d'une pièce"),
    ("Flottant", "Partie du capital en bourse", "Objet qui ne coule pas"),
    ("Masse", "Montant global (Masse salariale)", "Outil marteau"),
    ("Inflation", "Hausse des prix", "Gonflement d'un ballon"),
    ("Dépression", "Crise économique grave", "Maladie mentale"),
    ("Recette", "Rentrée d'argent public", "Instruction de cuisine"),
    ("Credit", "Autorisation de dépense (Crédit budgétaire)", "Confiance"),
    ("Enveloppe", "Budget alloué", "Papier pour lettre"),
    ("Cycle", "Fluctuation économique", "Vélo"),
    ("Taux", "Pourcentage fiscal", "Animal (Taupe mal écrit)")
]

def run_benchmark(model, dataset):
    correct = 0
    details = []

    # Encodage en batch pour la vitesse
    termes = [x[0] for x in dataset]
    bons = [x[1] for x in dataset]
    mauvais = [x[2] for x in dataset]

    emb_termes = model.encode(termes, convert_to_tensor=True)
    emb_bons = model.encode(bons, convert_to_tensor=True)
    emb_mauvais = model.encode(mauvais, convert_to_tensor=True)

    # Calculs
    sim_bons = util.cos_sim(emb_termes, emb_bons).diagonal()
    sim_mauvais = util.cos_sim(emb_termes, emb_mauvais).diagonal()

    for i in range(len(dataset)):
        score_bon = sim_bons[i].item()
        score_mauvais = sim_mauvais[i].item()

        is_correct = score_bon > score_mauvais
        if is_correct: correct += 1

        details.append({
            "Terme": termes[i],
            "Score_Bon": score_bon,
            "Score_Mauvais": score_mauvais,
            "Gagné": is_correct
        })

    return (correct / len(dataset)) * 100, pd.DataFrame(details)

score_snow, df_snow = run_benchmark(model_snowflake, dataset_jargon)
score_solon, df_solon = run_benchmark(model_solon, dataset_jargon)

print(f"\n📊 RÉSULTATS MÉTHODE 1 (Précision Jargon) :")
print(f"❄️ Snowflake : {score_snow:.2f}%")
print(f"🏛️ Solon     : {score_solon:.2f}%")

# Comparaison directe pour trouver les "Golden Samples"
df_snow["Model"] = "Snowflake"
df_solon["Model"] = "Solon"
df_merged = pd.merge(df_snow, df_solon, on="Terme", suffixes=('_Snow', '_Solon'))

# Les "Golden Samples" : Solon a bon, Snowflake a faux
golden = df_merged[ (df_merged["Gagné_Solon"] == True) & (df_merged["Gagné_Snow"] == False) ]

print(f"\n💎 GOLDEN SAMPLES ({len(golden)} termes trouvés) :")
print("Ce sont les termes que Solon comprend mieux que Snowflake :")
if not golden.empty:
    print(golden["Terme"].tolist())
else:
    print("Aucun ! Snowflake se défend très bien.")

🥊 LANCEMENT MÉTHODE 1 : TEST DE JARGON (60 TERMES)

📊 RÉSULTATS MÉTHODE 1 (Précision Jargon) :
❄️ Snowflake : 35.00%
🏛️ Solon     : 28.33%

💎 GOLDEN SAMPLES (5 termes trouvés) :
Ce sont les termes que Solon comprend mieux que Snowflake :
['Abattement', 'Provision', 'Arrêté', 'Disposition', 'Credit']


In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

print("⏳ Chargement des modèles Spécifiques...")
print("1. Téléchargement de Snowflake Arctic v2.0...")
try:
    # Le nouveau challenger généraliste puissant
    model_snowflake = SentenceTransformer('Snowflake/snowflake-arctic-embed-m-v2.0')
except Exception as e:
    print(f"⚠️ Erreur chargement Snowflake: {e}")
    # Fallback au cas où, mais normalement ça passe
    model_snowflake = SentenceTransformer('Snowflake/snowflake-arctic-embed-m')

print("2. Téléchargement de Solon Embeddings (Large 0.1)...")
try:
    # Le spécialiste du droit français par OrdalieTech
    model_solon = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')
except Exception as e:
    print(f"⚠️ Erreur chargement Solon: {e}")
    # Fallback sur un modèle juridique générique si Solon est inaccessible
    model_solon = SentenceTransformer('dangvantuan/sentence-camembert-base')

print("\n✅ Modèles chargés et prêts pour le duel !")

# --- LE DATASET "GRAND BERCY" (45 Termes) ---
dataset_bercy = [
    # --- Fiscalité des Particuliers ---
    ("TVA", "Impôt indirect sur la consommation payé par le consommateur final."),
    ("IR (Impôt sur le Revenu)", "Impôt direct calculé au niveau du foyer fiscal portant sur l'ensemble des revenus."),
    ("Prélèvement à la source", "Mode de recouvrement de l'impôt consistant à le faire payer au moment du versement du revenu."),
    ("Taxe d'habitation", "Impôt local dû par l'occupant d'un logement au 1er janvier de l'année."),
    ("Taxe Foncière", "Impôt local dû par les propriétaires de biens immobiliers."),
    ("PFU (Flat Tax)", "Prélèvement Forfaitaire Unique appliquant un taux fixe aux revenus du capital."),
    ("CSG (Contribution Sociale)", "Prélèvement obligatoire participant au financement de la sécurité sociale."),
    ("Droits de succession", "Impôt prélevé sur la transmission du patrimoine d'une personne décédée."),
    ("Niche fiscale", "Avantage fiscal dérogatoire permettant de diminuer le montant de l'impôt dû."),
    ("Quotient familial", "Système divisant le revenu imposable en nombre de parts selon la composition du foyer."),

    # --- Fiscalité des Entreprises ---
    ("IS (Impôt sur les Sociétés)", "Taxe prélevée sur le résultat annuel des entreprises exploitées en France."),
    ("CFE (Cotisation Foncière)", "Impôt local dû par les entreprises basé sur la valeur locative de leurs biens."),
    ("CVAE", "Cotisation calculée en fonction de la valeur ajoutée produite par l'entreprise."),
    ("Amortissement comptable", "Constatation comptable de la perte de valeur d'un actif du fait de l'usure ou du temps."),
    ("Bilan comptable", "Document synthétisant le patrimoine de l'entreprise à un instant T (Actif / Passif)."),
    ("Compte de résultat", "Document comptable présentant les produits et les charges d'un exercice."),
    ("BFR (Besoin en Fonds de Roulement)", "Mesure des ressources financières qu'une entreprise doit mobiliser pour couvrir le décalage de trésorerie."),
    ("Kbis", "Document officiel attestant de l'existence juridique d'une entreprise commerciale."),

    # --- Procédures et Contrôle Fiscal ---
    ("Assiette fiscale", "Montant servant de base au calcul d'un impôt ou d'une taxe."),
    ("Recouvrement", "Ensemble des procédures pour obtenir le paiement d'une somme due au Trésor public."),
    ("Rescrit fiscal", "Réponse formelle de l'administration à une question d'un contribuable sur sa situation, qui engage l'administration."),
    ("Dégrèvement", "Suppression ou atténuation partielle de l'impôt accordée par l'administration."),
    ("Redressement fiscal", "Opération de rectification par l'administration d'une imposition insuffisante ou omise."),
    ("Avis à tiers détenteur (ATD)", "Procédure permettant au Trésor de saisir une somme due par un tiers (banque, employeur) au débiteur."),
    ("Prescription fiscale", "Délai au-delà duquel l'administration ne peut plus réclamer l'impôt."),

    # --- Finances Publiques & LOLF ---
    ("Loi de Finances (PLF)", "Acte législatif prévoyant et autorisant le budget de l'État pour l'année à venir."),
    ("Déficit budgétaire", "Situation où les dépenses de l'État sont supérieures à ses recettes sur une année."),
    ("Dette publique", "Ensemble des engagements financiers cumulés pris par l'État et les administrations publiques."),
    ("BOP (Budget Opérationnel de Programme)", "Regroupement de crédits mis à la disposition d'un responsable pour mettre en œuvre un programme."),
    ("Engagement juridique", "Acte par lequel une personne morale publique crée ou constate une obligation de laquelle résultera une charge."),
    ("Liquidation (Dépense publique)", "Phase de la dépense consistant à vérifier la réalité de la dette et à arrêter le montant de la dépense."),
    ("Mandatement", "Acte par lequel l'ordonnateur donne l'ordre au comptable public de payer une dette."),
    ("Fongibilité des crédits", "Principe permettant d'utiliser des crédits prévus pour une nature de dépense pour une autre nature."),
    ("Titre de recette", "Acte administratif émis pour recouvrer une créance publique."),

    # --- Économie & Droit Administratif ---
    ("Marché public", "Contrat conclu à titre onéreux par un acheteur public pour répondre à ses besoins en matière de travaux, fournitures ou services."),
    ("Cadastre", "Registre public définissant l'emplacement, la surface et la valeur des propriétés foncières."),
    ("Plus-value", "Gain réalisé lors de la vente d'un bien à un prix supérieur à son prix d'achat."),
    ("PIB (Produit Intérieur Brut)", "Indicateur économique mesurant la richesse produite sur le territoire national."),
    ("Inflation", "Perte du pouvoir d'achat de la monnaie qui se traduit par une augmentation générale et durable des prix."),
    ("Obligation d'État (OAT)", "Titre de créance émis par l'État pour emprunter de l'argent sur les marchés financiers."),
    ("Taux directeur", "Taux d'intérêt fixé par la Banque Centrale influençant le coût du crédit bancaire."),
    ("Journal Officiel (JO)", "Publication quotidienne officielle où sont consignés les lois, décrets et arrêtés."),
    ("Décret", "Acte réglementaire ou individuel pris par le Président de la République ou le Premier ministre."),
    ("Jurisprudence", "Ensemble des décisions de justice qui servent de référence pour l'application du droit."),
    ("Contentieux administratif", "Ensemble des litiges relevant de la compétence des juridictions administratives.")
]

# Séparation termes/définitions
termes = [item[0] for item in dataset_bercy]
definitions = [item[1] for item in dataset_bercy]

# --- FONCTION D'ÉVALUATION ---
def evaluer_modele_global(model, model_name):
    print(f"--- 🚀 Évaluation de {model_name} ---")

    # Encodage vectoriel
    emb_termes = model.encode(termes, convert_to_tensor=True)
    emb_defs = model.encode(definitions, convert_to_tensor=True)

    # Calcul de similarité
    scores_matrix = util.cos_sim(emb_termes, emb_defs)

    top1_accuracy = 0
    top3_accuracy = 0
    erreurs = []

    for i, terme_reel in enumerate(termes):
        scores_du_terme = scores_matrix[i]

        # On regarde les 3 meilleures correspondances
        top_results = torch.topk(scores_du_terme, k=3)
        top_indices = top_results.indices.tolist()

        # Top 1 Check
        if top_indices[0] == i:
            top1_accuracy += 1
        else:
            faux_choix_index = top_indices[0]
            erreurs.append({
                "Terme": terme_reel,
                "A choisi (Erreur)": termes[faux_choix_index],
                "Score Faux": float(scores_du_terme[faux_choix_index]),
                "Score Vrai": float(scores_du_terme[i])
            })

        # Top 3 Check
        if i in top_indices:
            top3_accuracy += 1

    return {
        "name": model_name,
        "Top-1": (top1_accuracy / len(termes)) * 100,
        "Top-3": (top3_accuracy / len(termes)) * 100,
        "erreurs": erreurs
    }

# --- LANCEMENT ---
resultats_snow = evaluer_modele_global(model_snowflake, "Snowflake Arctic v2.0")
resultats_solon = evaluer_modele_global(model_solon, "Solon Embeddings (Large)")

# --- AFFICHAGE ---
summary = pd.DataFrame([
    {"Modèle": res["name"], "Top-1 (%)": f"{res['Top-1']:.1f}", "Top-3 (%)": f"{res['Top-3']:.1f}"}
    for res in [resultats_snow, resultats_solon]
])

print("\n📊 RÉSULTATS DU BENCHMARK 'GRAND BERCY' (45 Termes) 📊")
print(summary.to_markdown(index=False))

print("\n🕵️ ANALYSE DES PIÈGES (Où se sont-ils trompés ?)")
for res in [resultats_snow, resultats_solon]:
    print(f"\n> Modèle : {res['name']}")
    if not res["erreurs"]:
        print("   ✅ Aucune erreur ! (Performance parfaite)")
    else:
        # Tri des erreurs par 'Confiance' (Score Faux - Score Vrai)
        res["erreurs"].sort(key=lambda x: x["Score Faux"] - x["Score Vrai"], reverse=True)
        for err in res["erreurs"][:4]:
            diff = err['Score Faux'] - err['Score Vrai']
            print(f"   ⚠️ '{err['Terme']}' confondu avec '{err['A choisi (Erreur)']}' (Diff: {diff:.3f})")

⏳ Chargement des modèles Spécifiques...
1. Téléchargement de Snowflake Arctic v2.0...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


✅ Modèles chargés et prêts pour le duel !
--- 🚀 Évaluation de Snowflake Arctic v2.0 ---
--- 🚀 Évaluation de Solon Embeddings (Large) ---

📊 RÉSULTATS DU BENCHMARK 'GRAND BERCY' (45 Termes) 📊
| Modèle                   |   Top-1 (%) |   Top-3 (%) |
|:-------------------------|------------:|------------:|
| Snowflake Arctic v2.0    |        11.1 |        20   |
| Solon Embeddings (Large) |        62.2 |        77.8 |

🕵️ ANALYSE DES PIÈGES (Où se sont-ils trompés ?)

> Modèle : Snowflake Arctic v2.0
   ⚠️ 'Kbis' confondu avec 'Compte de résultat' (Diff: 0.114)
   ⚠️ 'IS (Impôt sur les Sociétés)' confondu avec 'Assiette fiscale' (Diff: 0.089)
   ⚠️ 'Taxe d'habitation' confondu avec 'Assiette fiscale' (Diff: 0.080)
   ⚠️ 'Redressement fiscal' confondu avec 'Niche fiscale' (Diff: 0.067)

> Modèle : Solon Embeddings (Large)
   ⚠️ 'Bilan comptable' confondu avec 'Compte de résultat' (Diff: 0.160)
   ⚠️ 'Kbis' confondu avec 'PIB (Produit Intérieur Brut)' (Diff: 0.129)
   ⚠️ 'Rescrit fiscal' c

bon code

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

# ==========================================
# 1. CHARGEMENT DES MODÈLES
# ==========================================
print("⏳ Chargement des modèles...")

# Modèle 1 : Snowflake Arctic v2.0 (Généraliste)
try:
    model_snowflake = SentenceTransformer('Snowflake/snowflake-arctic-embed-m-v2.0', trust_remote_code=True)
except:
    print("⚠️ Snowflake v2.0 échec, passage sur v1.5 ou fallback...")
    model_snowflake = SentenceTransformer('Snowflake/snowflake-arctic-embed-m')

# Modèle 2 : Solon (Spécialiste Juridique)
try:
    model_solon = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1')
except:
    model_solon = SentenceTransformer('dangvantuan/sentence-camembert-base')

print("✅ Modèles prêts.")

# ==========================================
# 2. DATASET PRINCIPAL (Les cibles)
# ==========================================
# Ce sont les concepts que l'IA doit trouver.
dataset_termes = [
    "TVA", "Impôt sur le Revenu", "Prélèvement à la source", "Taxe d'habitation", "Taxe Foncière",
    "PFU (Flat Tax)", "CSG", "Droits de succession", "Niche fiscale", "Quotient familial",
    "Impôt sur les Sociétés (IS)", "CFE", "CVAE", "Amortissement comptable", "Bilan comptable",
    "Compte de résultat", "BFR", "Kbis", "Assiette fiscale", "Recouvrement",
    "Rescrit fiscal", "Dégrèvement", "Redressement fiscal", "Avis à tiers détenteur", "Prescription fiscale",
    "Loi de Finances", "Déficit budgétaire", "Dette publique", "BOP", "Engagement juridique",
    "Liquidation", "Mandatement", "Fongibilité", "Titre de recette", "Marché public",
    "Cadastre", "Plus-value", "PIB", "Inflation", "OAT (Obligation État)",
    "Taux directeur", "Journal Officiel", "Décret", "Jurisprudence", "Contentieux administratif"
]

# ==========================================
# 3. NOUVEAU DATASET : QUESTIONS / RÉPONSES (Q&A)
# ==========================================
# L'IA doit trouver le bon terme (ci-dessus) à partir d'une question d'un usager.
dataset_qa = [
    ("Quel est l'impôt que je paie sur ce que je gagne chaque mois en tant que salarié ?", "Impôt sur le Revenu"),
    ("Je viens d'acheter un chewing-gum, quelle taxe est incluse dans le prix ?", "TVA"),
    ("L'État me réclame de l'argent après la mort de mon oncle, c'est quoi ?", "Droits de succession"),
    ("Je suis propriétaire de mon appartement, que dois-je payer en octobre ?", "Taxe Foncière"),
    ("Comment s'appelle le document qui prouve que ma société est bien immatriculée ?", "Kbis"),
    ("J'ai fait des bénéfices avec ma SAS, quel impôt la société doit-elle payer ?", "Impôt sur les Sociétés (IS)"),
    ("L'administration a fait une erreur et annule mon impôt, comment ça s'appelle ?", "Dégrèvement"),
    ("L'État dépense plus qu'il ne gagne cette année, c'est quelle situation ?", "Déficit budgétaire"),
    ("Je veux consulter les lois votées hier, je regarde où ?", "Journal Officiel"),
    ("La banque centrale augmente les taux, de quoi parle-t-on ?", "Taux directeur"),
    ("C'est quoi le total des dettes de la France ?", "Dette publique"),
    ("Le fisc peut saisir mon salaire directement chez mon employeur, c'est quelle procédure ?", "Avis à tiers détenteur"),
    ("Au bout de combien de temps le fisc ne peut plus me réclamer d'argent ?", "Prescription fiscale"),
    ("C'est quoi la différence entre mes actifs et mes passifs dans ma compta ?", "Bilan comptable"),
    ("Quel indicateur montre la richesse créée par le pays en un an ?", "PIB"),
    ("Les prix augmentent partout, c'est quel phénomène ?", "Inflation"),
    ("Je veux savoir à qui appartient le terrain voisin, je consulte quoi ?", "Cadastre"),
    ("J'ai revendu ma maison plus cher que je l'ai achetée, j'ai fait quoi ?", "Plus-value"),
    ("Comment s'appelle l'acte officiel qui valide le budget de l'État pour l'an prochain ?", "Loi de Finances"),
    ("C'est quoi la réponse écrite du fisc qui le lie juridiquement ?", "Rescrit fiscal")
]

# ==========================================
# 4. NOUVEAU DATASET : TRIPLETS (Anchor, Positive, Negative)
# ==========================================
# L'IA doit trouver que l'Ancre est plus proche du Positif que du Négatif.
# Format: (Ancre, Concept Proche, Concept Éloigné/Piège)
dataset_triplets = [
    ("TVA", "Consommation", "Revenu"), # La TVA taxe la conso, pas le revenu
    ("Bilan comptable", "Patrimoine", "Rentabilité"), # Bilan = Stock/Patrimoine, Compte résultat = Flux/Rentabilité
    ("IS (Impôt Sociétés)", "Bénéfice", "Chiffre d'affaires"), # L'IS est sur le bénéfice, la CVAE/TVA sur le CA
    ("Kbis", "Identité entreprise", "Bilan financier"),
    ("Recouvrement", "Trésor Public", "Dépense publique"), # Recouvrement = faire entrer l'argent
    ("Loi de Finances", "Parlement", "Entreprise"), # Voté par le parlement
    ("Mandatement", "Paiement", "Recette"), # Mandatement = étape de la dépense (paiement)
    ("Cadastre", "Immobilier", "Action en bourse"),
    ("Inflation", "Hausse des prix", "Croissance"),
    ("Jurisprudence", "Juge", "Député"), # Le juge fait la jurisprudence, le député la loi
    ("Décret", "Premier Ministre", "Maire"),
    ("Taxe Foncière", "Propriétaire", "Locataire"),
    ("CSG", "Sécurité Sociale", "Budget de l'État"), # La CSG finance la sécu
    ("Amortissement", "Usure", "Trésorerie"),
    ("Marché public", "Appel d'offres", "Contrat privé"),
    ("Rescrit fiscal", "Garantie", "Contrôle"),
    ("Plus-value", "Vente", "Achat"),
    ("OAT", "Emprunt", "Impôt"),
    ("Solde budgétaire", "Déficit", "Bilan comptable"),
    ("Redressement", "Rectification", "Réduction")
]

# ==========================================
# 5. FONCTIONS D'ÉVALUATION
# ==========================================

def eval_qa(model, name):
    """Teste si l'IA retrouve le bon terme technique à partir de la question"""
    # 1. On vectorise tous les termes cibles (la "base de connaissance")
    emb_termes = model.encode(dataset_termes, convert_to_tensor=True)

    questions = [q[0] for q in dataset_qa]
    bonnes_reponses_txt = [q[1] for q in dataset_qa]
    emb_questions = model.encode(questions, convert_to_tensor=True)

    # Matrice de similarité
    hits = 0
    scores_matrix = util.cos_sim(emb_questions, emb_termes)

    for i, _ in enumerate(questions):
        # Le terme attendu pour cette question
        target_term = bonnes_reponses_txt[i]
        try:
            target_idx = dataset_termes.index(target_term)
        except ValueError:
            continue # Sécurité si faute de frappe

        # Les prédictions du modèle
        scores = scores_matrix[i]
        top_result_idx = torch.argmax(scores).item()

        # Est-ce que le modèle a trouvé le bon index ?
        if top_result_idx == target_idx:
            hits += 1

    return (hits / len(dataset_qa)) * 100

def eval_triplets(model, name):
    """Teste si le modèle rapproche l'ancre du positif et l'éloigne du négatif"""
    ancres = [t[0] for t in dataset_triplets]
    positifs = [t[1] for t in dataset_triplets]
    negatifs = [t[2] for t in dataset_triplets]

    emb_a = model.encode(ancres, convert_to_tensor=True)
    emb_p = model.encode(positifs, convert_to_tensor=True)
    emb_n = model.encode(negatifs, convert_to_tensor=True)

    # Calcul des distances (Similarité Cosinus)
    # On veut sim(A, P) > sim(A, N)
    sim_ap = torch.nn.functional.cosine_similarity(emb_a, emb_p)
    sim_an = torch.nn.functional.cosine_similarity(emb_a, emb_n)

    success = 0
    erreurs = []

    for i in range(len(dataset_triplets)):
        score_pos = sim_ap[i].item()
        score_neg = sim_an[i].item()

        if score_pos > score_neg:
            success += 1
        else:
            erreurs.append({
                "Ancre": ancres[i],
                "A préféré (Erreur)": negatifs[i],
                "Au lieu de": positifs[i],
                "Score Faux": score_neg,
                "Score Vrai": score_pos
            })

    return (success / len(dataset_triplets)) * 100, erreurs

# ==========================================
# 6. EXÉCUTION DU BENCHMARK COMPLET
# ==========================================

results = []

for model, nom in [(model_snowflake, "Snowflake Arctic v2"), (model_solon, "Solon Embeddings")]:
    print(f"\n--- Test de {nom} ---")

    # Test 1 : Q&A (Compréhension naturelle)
    score_qa = eval_qa(model, nom)
    print(f"   > Score Q&A : {score_qa:.1f}%")

    # Test 2 : Triplets (Logique & Nuance)
    score_triplets, errs = eval_triplets(model, nom)
    print(f"   > Score Triplets : {score_triplets:.1f}%")

    results.append({
        "Modèle": nom,
        "Q&A Accuracy": score_qa,
        "Triplets Accuracy": score_triplets
    })

    # Affichage des erreurs de triplets pour comprendre
    if errs:
        print("   ⚠️ Exemples d'erreurs de logique (Triplets) :")
        for e in errs[:3]: # On en montre 3
            print(f"      - Pour '{e['Ancre']}', il trouve '{e['A préféré (Erreur)']}' plus proche que '{e['Au lieu de']}'")

# ==========================================
# 7. TABLEAU FINAL
# ==========================================
df_res = pd.DataFrame(results)
print("\n🏆 RÉSULTATS FINAUX (Q&A + TRIPLETS) 🏆")
print(df_res.to_markdown(index=False))

⏳ Chargement des modèles...


configuration_hf_alibaba_nlp_gte.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Snowflake/snowflake-arctic-embed-m-v2.0:
- configuration_hf_alibaba_nlp_gte.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_hf_alibaba_nlp_gte.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Snowflake/snowflake-arctic-embed-m-v2.0:
- modeling_hf_alibaba_nlp_gte.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

⚠️ Snowflake v2.0 échec, passage sur v1.5 ou fallback...
✅ Modèles prêts.

--- Test de Snowflake Arctic v2 ---
   > Score Q&A : 15.0%
   > Score Triplets : 50.0%
   ⚠️ Exemples d'erreurs de logique (Triplets) :
      - Pour 'Bilan comptable', il trouve 'Rentabilité' plus proche que 'Patrimoine'
      - Pour 'IS (Impôt Sociétés)', il trouve 'Chiffre d'affaires' plus proche que 'Bénéfice'
      - Pour 'Recouvrement', il trouve 'Dépense publique' plus proche que 'Trésor Public'

--- Test de Solon Embeddings ---
   > Score Q&A : 55.0%
   > Score Triplets : 50.0%
   ⚠️ Exemples d'erreurs de logique (Triplets) :
      - Pour 'TVA', il trouve 'Revenu' plus proche que 'Consommation'
      - Pour 'Bilan comptable', il trouve 'Rentabilité' plus proche que 'Patrimoine'
      - Pour 'IS (Impôt Sociétés)', il trouve 'Chiffre d'affaires' plus proche que 'Bénéfice'

🏆 RÉSULTATS FINAUX (Q&A + TRIPLETS) 🏆
| Modèle              |   Q&A Accuracy |   Triplets Accuracy |
|:--------------------|------------

In [ ]:
import pandas as pd
import torch
import gc # Garbage Collector pour libérer la RAM
from sentence_transformers import SentenceTransformer, util

# ==========================================
# 1. LISTE DES MODÈLES À COMPARER
# ==========================================
# Note: "bge-m3-unsupervised" n'existant pas en repo distinct sur HF (c'est le même modèle),
# j'ai gardé BGE-M3 standard qui inclut ces capacités.

models_config = [
    {"name": "Solon (Juridique FR)", "id": "OrdalieTech/Solon-embeddings-large-0.1"},
    {"name": "Snowflake Arctic v2.0", "id": "Snowflake/snowflake-arctic-embed-m-v2.0"},
    {"name": "BGE-M3 (Multilingue)", "id": "BAAI/bge-m3"},
    {"name": "GTE-Multilingual Base", "id": "Alibaba-NLP/gte-multilingual-base"},
    {"name": "E5-Large Instruct", "id": "intfloat/multilingual-e5-large-instruct"},
    {"name": "UAE-Large-V1", "id": "WhereIsAI/UAE-Large-V1"},
    # BGE-M3 est déjà listé au-dessus, inutile de le charger deux fois.
]

# ==========================================
# 2. DATASETS (GRAND BERCY + Q&A + TRIPLETS)
# ==========================================

dataset_bercy = [
    ("TVA", "Impôt indirect sur la consommation payé par le consommateur final."),
    ("IR (Impôt sur le Revenu)", "Impôt direct calculé au niveau du foyer fiscal portant sur l'ensemble des revenus."),
    ("Prélèvement à la source", "Mode de recouvrement de l'impôt consistant à le faire payer au moment du versement du revenu."),
    ("Taxe d'habitation", "Impôt local dû par l'occupant d'un logement au 1er janvier de l'année."),
    ("Taxe Foncière", "Impôt local dû par les propriétaires de biens immobiliers."),
    ("PFU (Flat Tax)", "Prélèvement Forfaitaire Unique appliquant un taux fixe aux revenus du capital."),
    ("CSG (Contribution Sociale)", "Prélèvement obligatoire participant au financement de la sécurité sociale."),
    ("Droits de succession", "Impôt prélevé sur la transmission du patrimoine d'une personne décédée."),
    ("Niche fiscale", "Avantage fiscal dérogatoire permettant de diminuer le montant de l'impôt dû."),
    ("Quotient familial", "Système divisant le revenu imposable en nombre de parts selon la composition du foyer."),
    ("IS (Impôt sur les Sociétés)", "Taxe prélevée sur le résultat annuel des entreprises exploitées en France."),
    ("CFE (Cotisation Foncière)", "Impôt local dû par les entreprises basé sur la valeur locative de leurs biens."),
    ("CVAE", "Cotisation calculée en fonction de la valeur ajoutée produite par l'entreprise."),
    ("Amortissement comptable", "Constatation comptable de la perte de valeur d'un actif du fait de l'usure ou du temps."),
    ("Bilan comptable", "Document synthétisant le patrimoine de l'entreprise à un instant T (Actif / Passif)."),
    ("Compte de résultat", "Document comptable présentant les produits et les charges d'un exercice."),
    ("BFR (Besoin en Fonds de Roulement)", "Mesure des ressources financières qu'une entreprise doit mobiliser pour couvrir le décalage de trésorerie."),
    ("Kbis", "Document officiel attestant de l'existence juridique d'une entreprise commerciale."),
    ("Assiette fiscale", "Montant servant de base au calcul d'un impôt ou d'une taxe."),
    ("Recouvrement", "Ensemble des procédures pour obtenir le paiement d'une somme due au Trésor public."),
    ("Rescrit fiscal", "Réponse formelle de l'administration à une question d'un contribuable sur sa situation, qui engage l'administration."),
    ("Dégrèvement", "Suppression ou atténuation partielle de l'impôt accordée par l'administration."),
    ("Redressement fiscal", "Opération de rectification par l'administration d'une imposition insuffisante ou omise."),
    ("Avis à tiers détenteur (ATD)", "Procédure permettant au Trésor de saisir une somme due par un tiers (banque, employeur) au débiteur."),
    ("Prescription fiscale", "Délai au-delà duquel l'administration ne peut plus réclamer l'impôt."),
    ("Loi de Finances (PLF)", "Acte législatif prévoyant et autorisant le budget de l'État pour l'année à venir."),
    ("Déficit budgétaire", "Situation où les dépenses de l'État sont supérieures à ses recettes sur une année."),
    ("Dette publique", "Ensemble des engagements financiers cumulés pris par l'État et les administrations publiques."),
    ("BOP (Budget Opérationnel de Programme)", "Regroupement de crédits mis à la disposition d'un responsable pour mettre en œuvre un programme."),
    ("Engagement juridique", "Acte par lequel une personne morale publique crée ou constate une obligation de laquelle résultera une charge."),
    ("Liquidation (Dépense publique)", "Phase de la dépense consistant à vérifier la réalité de la dette et à arrêter le montant de la dépense."),
    ("Mandatement", "Acte par lequel l'ordonnateur donne l'ordre au comptable public de payer une dette."),
    ("Fongibilité des crédits", "Principe permettant d'utiliser des crédits prévus pour une nature de dépense pour une autre nature."),
    ("Titre de recette", "Acte administratif émis pour recouvrer une créance publique."),
    ("Marché public", "Contrat conclu à titre onéreux par un acheteur public pour répondre à ses besoins en matière de travaux, fournitures ou services."),
    ("Cadastre", "Registre public définissant l'emplacement, la surface et la valeur des propriétés foncières."),
    ("Plus-value", "Gain réalisé lors de la vente d'un bien à un prix supérieur à son prix d'achat."),
    ("PIB (Produit Intérieur Brut)", "Indicateur économique mesurant la richesse produite sur le territoire national."),
    ("Inflation", "Perte du pouvoir d'achat de la monnaie qui se traduit par une augmentation générale et durable des prix."),
    ("Obligation d'État (OAT)", "Titre de créance émis par l'État pour emprunter de l'argent sur les marchés financiers."),
    ("Taux directeur", "Taux d'intérêt fixé par la Banque Centrale influençant le coût du crédit bancaire."),
    ("Journal Officiel (JO)", "Publication quotidienne officielle où sont consignés les lois, décrets et arrêtés."),
    ("Décret", "Acte réglementaire ou individuel pris par le Président de la République ou le Premier ministre."),
    ("Jurisprudence", "Ensemble des décisions de justice qui servent de référence pour l'application du droit."),
    ("Contentieux administratif", "Ensemble des litiges relevant de la compétence des juridictions administratives.")
]

dataset_qa = [
    ("Quel est l'impôt que je paie sur ce que je gagne chaque mois en tant que salarié ?", "Impôt sur le Revenu"),
    ("Je viens d'acheter un chewing-gum, quelle taxe est incluse dans le prix ?", "TVA"),
    ("L'État me réclame de l'argent après la mort de mon oncle, c'est quoi ?", "Droits de succession"),
    ("Je suis propriétaire de mon appartement, que dois-je payer en octobre ?", "Taxe Foncière"),
    ("Comment s'appelle le document qui prouve que ma société est bien immatriculée ?", "Kbis"),
    ("J'ai fait des bénéfices avec ma SAS, quel impôt la société doit-elle payer ?", "Impôt sur les Sociétés (IS)"),
    ("L'administration a fait une erreur et annule mon impôt, comment ça s'appelle ?", "Dégrèvement"),
    ("L'État dépense plus qu'il ne gagne cette année, c'est quelle situation ?", "Déficit budgétaire"),
    ("Je veux consulter les lois votées hier, je regarde où ?", "Journal Officiel"),
    ("La banque centrale augmente les taux, de quoi parle-t-on ?", "Taux directeur"),
    ("C'est quoi le total des dettes de la France ?", "Dette publique"),
    ("Le fisc peut saisir mon salaire directement chez mon employeur, c'est quelle procédure ?", "Avis à tiers détenteur"),
    ("Au bout de combien de temps le fisc ne peut plus me réclamer d'argent ?", "Prescription fiscale"),
    ("C'est quoi la différence entre mes actifs et mes passifs dans ma compta ?", "Bilan comptable"),
    ("Quel indicateur montre la richesse créée par le pays en un an ?", "PIB"),
    ("Les prix augmentent partout, c'est quel phénomène ?", "Inflation"),
    ("Je veux savoir à qui appartient le terrain voisin, je consulte quoi ?", "Cadastre"),
    ("J'ai revendu ma maison plus cher que je l'ai achetée, j'ai fait quoi ?", "Plus-value"),
    ("Comment s'appelle l'acte officiel qui valide le budget de l'État pour l'an prochain ?", "Loi de Finances"),
    ("C'est quoi la réponse écrite du fisc qui le lie juridiquement ?", "Rescrit fiscal")
]

dataset_triplets = [
    ("TVA", "Consommation", "Revenu"),
    ("Bilan comptable", "Patrimoine", "Rentabilité"),
    ("IS (Impôt Sociétés)", "Bénéfice", "Chiffre d'affaires"),
    ("Kbis", "Identité entreprise", "Bilan financier"),
    ("Recouvrement", "Trésor Public", "Dépense publique"),
    ("Loi de Finances", "Parlement", "Entreprise"),
    ("Mandatement", "Paiement", "Recette"),
    ("Cadastre", "Immobilier", "Action en bourse"),
    ("Inflation", "Hausse des prix", "Croissance"),
    ("Jurisprudence", "Juge", "Député"),
    ("Décret", "Premier Ministre", "Maire"),
    ("Taxe Foncière", "Propriétaire", "Locataire"),
    ("CSG", "Sécurité Sociale", "Budget de l'État"),
    ("Amortissement", "Usure", "Trésorerie"),
    ("Marché public", "Appel d'offres", "Contrat privé"),
    ("Rescrit fiscal", "Garantie", "Contrôle"),
    ("Plus-value", "Vente", "Achat"),
    ("OAT", "Emprunt", "Impôt"),
    ("Solde budgétaire", "Déficit", "Bilan comptable"),
    ("Redressement", "Rectification", "Réduction")
]

# Séparation des listes pour le test 1
termes_bercy = [item[0] for item in dataset_bercy]
defs_bercy = [item[1] for item in dataset_bercy]

# ==========================================
# 3. MOTEUR D'ÉVALUATION (Optimisé)
# ==========================================

def run_full_benchmark(model, model_name):
    """
    Exécute les 3 tests (Defs, QA, Triplets) pour un modèle donné.
    Gère les préfixes pour les modèles type E5.
    """

    # ---------------------------------------------
    # GESTION DES INSTRUCTIONS (PROMPTS)
    # E5 nécessite "query: " pour les questions/ancres
    # ---------------------------------------------
    prefix_q = ""
    prefix_d = ""
    if "e5" in model_name.lower() or "instruct" in model_name.lower():
        prefix_q = "query: "
        prefix_d = "passage: "

    print(f"   🔹 Encodage en cours (Mode: {'Instruct' if prefix_q else 'Standard'})...")

    # --- TEST 1 : DÉFINITIONS (Matching) ---
    # On ajoute les préfixes si nécessaire
    inputs_termes = [prefix_q + t for t in termes_bercy]
    inputs_defs = [prefix_d + d for d in defs_bercy]

    emb_termes = model.encode(inputs_termes, convert_to_tensor=True, show_progress_bar=False)
    emb_defs = model.encode(inputs_defs, convert_to_tensor=True, show_progress_bar=False)

    scores = util.cos_sim(emb_termes, emb_defs)
    top1 = 0
    top3 = 0

    for i in range(len(termes_bercy)):
        top_indices = torch.topk(scores[i], k=3).indices.tolist()
        if top_indices[0] == i: top1 += 1
        if i in top_indices: top3 += 1

    score_defs_top1 = (top1 / len(termes_bercy)) * 100
    score_defs_top3 = (top3 / len(termes_bercy)) * 100

    # --- TEST 2 : Q&A (Retrieval) ---
    q_txt = [prefix_q + x[0] for x in dataset_qa]
    # Les cibles (termes) sont déjà encodés dans emb_termes, on réutilise !
    emb_q = model.encode(q_txt, convert_to_tensor=True, show_progress_bar=False)

    qa_hits = 0
    qa_scores = util.cos_sim(emb_q, emb_termes)

    for i, item in enumerate(dataset_qa):
        target_term = item[1]
        try:
            target_idx = termes_bercy.index(target_term)
            if torch.argmax(qa_scores[i]).item() == target_idx:
                qa_hits += 1
        except ValueError:
            continue

    score_qa = (qa_hits / len(dataset_qa)) * 100

    # --- TEST 3 : TRIPLETS (Logique) ---
    ancres = [prefix_q + t[0] for t in dataset_triplets]
    positifs = [prefix_d + t[1] for t in dataset_triplets]
    negatifs = [prefix_d + t[2] for t in dataset_triplets]

    emb_a = model.encode(ancres, convert_to_tensor=True, show_progress_bar=False)
    emb_p = model.encode(positifs, convert_to_tensor=True, show_progress_bar=False)
    emb_n = model.encode(negatifs, convert_to_tensor=True, show_progress_bar=False)

    sim_ap = torch.nn.functional.cosine_similarity(emb_a, emb_p)
    sim_an = torch.nn.functional.cosine_similarity(emb_a, emb_n)

    triplet_success = 0
    for i in range(len(dataset_triplets)):
        if sim_ap[i] > sim_an[i]:
            triplet_success += 1

    score_triplets = (triplet_success / len(dataset_triplets)) * 100

    return {
        "Modèle": model_name,
        "Déf. Top-1": score_defs_top1,
        "Déf. Top-3": score_defs_top3,
        "Q&A": score_qa,
        "Triplets": score_triplets,
        "Moyenne Globale": (score_defs_top1 + score_qa + score_triplets) / 3
    }

# ==========================================
# 4. BOUCLE PRINCIPALE (Batch Processing)
# ==========================================
all_results = []

print(f"🚀 DÉMARRAGE DU GRAND BENCHMARK ({len(models_config)} modèles)")

for config in models_config:
    nom = config["name"]
    repo = config["id"]

    print(f"\n📥 Chargement de : {nom} ...")
    try:
        # Chargement avec trust_remote_code pour les modèles récents (Snowflake, BGE)
        model = SentenceTransformer(repo, trust_remote_code=True)

        # Exécution des tests
        res = run_full_benchmark(model, nom)
        all_results.append(res)

        print(f"   ✅ Terminé. Score Global: {res['Moyenne Globale']:.1f}%")

        # NETTOYAGE MÉMOIRE (CRUCIAL)
        del model
        gc.collect()
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"   ❌ Échec chargement {nom}: {e}")
        all_results.append({"Modèle": nom, "Moyenne Globale": 0.0})

# ==========================================
# 5. AFFICHAGE FINAL
# ==========================================
df_final = pd.DataFrame(all_results)
# On trie par Score Global décroissant
df_final = df_final.sort_values(by="Moyenne Globale", ascending=False)

print("\n🏆 CLASSEMENT FINAL : IA vs BERCY 🏆")
print(df_final.to_markdown(index=False, floatfmt=".1f"))

🚀 DÉMARRAGE DU GRAND BENCHMARK (6 modèles)

📥 Chargement de : Solon (Juridique FR) ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

   🔹 Encodage en cours (Mode: Standard)...
   ✅ Terminé. Score Global: 28.3%

🏆 CLASSEMENT FINAL : IA vs BERCY 🏆
| Modèle                |   Déf. Top-1 |   Déf. Top-3 |   Q&A |   Triplets |   Moyenne Globale |
|:----------------------|-------------:|-------------:|------:|-----------:|------------------:|
| E5-Large Instruct     |         60.0 |         86.7 |  25.0 |       70.0 |              51.7 |
| Solon (Juridique FR)  |         62.2 |         77.8 |  35.0 |       50.0 |              49.1 |
| GTE-Multilingual Base |         42.2 |         60.0 |  25.0 |       70.0 |              45.7 |
| BGE-M3 (Multilingue)  |         35.6 |         62.2 |  25.0 |       55.0 |              38.5 |
| UAE-Large-V1          |         20.0 |         37.8 |   5.0 |       60.0 |              28.3 |
| Snowflake Arctic v2.0 |        nan   |        nan   | nan   |      nan   |               0.0 |


In [ ]:
import pandas as pd
import torch
import gc
from sentence_transformers import SentenceTransformer, util

# ==========================================
# 1. LE "MEGA DATASET" BERCY (85+ Termes)
# ==========================================
# Structure : Fiscalité, Compta Publique (GBCP), Douanes, Marchés Publics, Macro.

dataset_bercy = [
    # --- FISCALITÉ PART & PRO ---
    ("TVA", "Impôt indirect sur la consommation payé par le consommateur final."),
    ("TVA Intracommunautaire", "Numéro d'identification fiscal individuel pour les échanges au sein de l'UE."),
    ("IR (Impôt sur le Revenu)", "Impôt direct progressif calculé au niveau du foyer fiscal sur l'ensemble des revenus."),
    ("Prélèvement à la source", "Mode de recouvrement de l'impôt consistant à le faire payer au moment du versement du revenu."),
    ("Taxe d'habitation", "Impôt local dû par l'occupant d'un logement au 1er janvier (résidence secondaire)."),
    ("Taxe Foncière", "Impôt local dû par les propriétaires de biens immobiliers bâtis ou non bâtis."),
    ("PFU (Flat Tax)", "Prélèvement Forfaitaire Unique appliquant un taux fixe (30%) aux revenus du capital."),
    ("CSG", "Prélèvement obligatoire participant au financement de la sécurité sociale (maladie, famille, retraite)."),
    ("CRDS", "Contribution pour le Remboursement de la Dette Sociale."),
    ("Droits de succession", "Impôt prélevé sur la transmission du patrimoine d'une personne décédée."),
    ("Droits de mutation (DMTO)", "Taxes perçues par le notaire lors d'un transfert de propriété immobilière."),
    ("Niche fiscale", "Avantage fiscal dérogatoire (crédit, réduction) permettant de diminuer l'impôt dû."),
    ("Quotient familial", "Système divisant le revenu imposable en nombre de parts selon la composition du foyer."),
    ("IS (Impôt sur les Sociétés)", "Impôt prélevé sur le bénéfice annuel des entreprises exploitées en France."),
    ("CFE", "Cotisation Foncière des Entreprises, basée sur la valeur locative des biens passibles de taxe foncière."),
    ("CVAE", "Cotisation sur la Valeur Ajoutée des Entreprises, due par les entreprises dépassant un certain CA."),
    ("TASCOM", "Taxe sur les surfaces commerciales due par les magasins de commerce de détail."),
    ("TGAP", "Taxe Générale sur les Activités Polluantes."),
    ("Accises", "Impôts indirects portant sur la consommation de certains produits (alcool, tabac, pétrole)."),
    ("Octroi de mer", "Taxe perçue sur les produits importés dans les départements d'outre-mer."),

    # --- COMPTABILITÉ & FINANCE D'ENTREPRISE ---
    ("Amortissement comptable", "Constatation comptable de la perte de valeur définitive d'un actif (usure, temps)."),
    ("Provision", "Passif dont l'échéance ou le montant est incertain, destiné à couvrir un risque probable."),
    ("Bilan comptable", "Document synthétisant le patrimoine de l'entreprise à un instant T (Actif / Passif)."),
    ("Compte de résultat", "Document comptable présentant les produits et les charges d'un exercice pour déterminer le résultat."),
    ("Annexe comptable", "Document expliquant et détaillant les chiffres du bilan et du compte de résultat."),
    ("BFR (Besoin en Fonds de Roulement)", "Décalage de trésorerie né du cycle d'exploitation (Stocks + Créances - Dettes)."),
    ("CAF (Capacité d'Autofinancement)", "Flux de trésorerie potentiel généré par l'activité de l'entreprise."),
    ("EBE (Excédent Brut d'Exploitation)", "Ressource générée par l'exploitation avant prise en compte des amortissements et frais financiers."),
    ("Kbis", "Document officiel attestant de l'existence juridique d'une entreprise commerciale."),
    ("SIREN", "Numéro unique d'identification de l'entreprise (9 chiffres)."),
    ("SIRET", "Numéro d'identification de chaque établissement de l'entreprise (14 chiffres)."),
    ("Liasses fiscales", "Ensemble des déclarations fiscales remises par les entreprises à l'administration."),

    # --- PROCÉDURES & CONTRÔLE FISCAL ---
    ("Assiette fiscale", "Montant (base) servant de calcul à un impôt ou une taxe."),
    ("Liquidation (Impôt)", "Calcul du montant de l'impôt exigible une fois l'assiette déterminée."),
    ("Recouvrement", "Ensemble des procédures pour obtenir le paiement d'une somme due au Trésor public."),
    ("Rescrit fiscal", "Réponse formelle de l'administration à une question d'un contribuable, qui l'engage juridiquement."),
    ("Dégrèvement", "Suppression ou atténuation partielle de l'impôt accordée par l'administration (ex: erreur)."),
    ("Redressement fiscal", "Opération de rectification par l'administration d'une imposition insuffisante ou omise."),
    ("Avis à tiers détenteur (ATD)", "Procédure de saisie d'une somme due par un tiers (banque, employeur) au débiteur de l'impôt."),
    ("Prescription fiscale", "Délai au-delà duquel l'administration ne peut plus contrôler ou réclamer l'impôt."),
    ("Contrôle sur pièces", "Vérification de la cohérence des déclarations effectuée depuis les bureaux de l'administration."),
    ("Vérification de comptabilité", "Contrôle fiscal effectué dans les locaux de l'entreprise."),
    ("ESFP", "Examen de la Situation Fiscale Personnelle d'un particulier."),
    ("Fraude fiscale", "Soustraction illégale au paiement de l'impôt (manœuvres frauduleuses)."),
    ("Optimisation fiscale", "Utilisation des moyens légaux pour réduire la charge fiscale."),
    ("Abus de droit", "Montage juridique fictif ou ayant pour seul but d'éluder l'impôt."),

    # --- FINANCES PUBLIQUES & GESTION (GBCP/LOLF) ---
    ("Loi de Finances (PLF)", "Acte législatif prévoyant et autorisant le budget de l'État pour l'année à venir."),
    ("Loi de Règlement", "Loi constatant les résultats financiers de l'année civile écoulée."),
    ("Déficit budgétaire", "Solde négatif du budget de l'État (Dépenses > Recettes)."),
    ("Dette publique", "Ensemble des engagements financiers cumulés de l'État et des administrations publiques."),
    ("BOP (Budget Opérationnel de Programme)", "Regroupement de crédits mis à disposition d'un responsable local."),
    ("AE (Autorisations d'Engagement)", "Limite supérieure des dépenses pouvant être engagées (juridiquement)."),
    ("CP (Crédits de Paiement)", "Limite supérieure des dépenses pouvant être payées (décaissées) dans l'année."),
    ("Engagement juridique", "Acte par lequel une personne publique crée une obligation (bon de commande)."),
    ("Service fait", "Certification que la prestation a été réalisée, conditionnant le paiement."),
    ("Liquidation (Dépense)", "Vérification de la réalité de la dette et calcul exact du montant à payer."),
    ("Mandatement", "Ordre donné par l'ordonnateur au comptable de payer une dépense."),
    ("Ordonnateur", "Agent public qui décide de la dépense mais ne manie pas les fonds."),
    ("Comptable public", "Agent public seul habilité à manier les fonds et payer les dépenses."),
    ("Principe de séparation", "Règle interdisant d'être à la fois ordonnateur (décideur) et comptable (payeur)."),
    ("Fongibilité des crédits", "Possibilité d'utiliser des crédits d'un titre pour un autre (sauf personnel)."),
    ("Titre de recette", "Acte administratif émis pour recouvrer une créance publique."),
    ("Reste à recouvrer (RAR)", "Sommes dues à l'État non encore encaissées."),
    ("Admission en non-valeur", "Décision d'arrêter les poursuites de recouvrement (dette irrécouvrable)."),
    ("Chorus Pro", "Portail de facturation électronique obligatoire pour la sphère publique."),

    # --- MARCHÉS PUBLICS & DROIT ADM ---
    ("Marché public", "Contrat onéreux conclu par un acheteur public pour répondre à ses besoins."),
    ("Accord-cadre", "Contrat fixant les termes régissant les marchés à passer au cours d'une période."),
    ("Appel d'offres", "Procédure où l'acheteur choisit l'offre économiquement la plus avantageuse sans négociation."),
    ("Sourcing", "Consultation préalable des entreprises pour définir le besoin avant un marché."),
    ("DUME", "Document Unique de Marché Européen (déclaration sur l'honneur)."),
    ("CCAG", "Cahier des Clauses Administratives Générales (conditions standards)."),
    ("CCTP", "Cahier des Clauses Techniques Particulières (spécifications techniques)."),
    ("Recours gracieux", "Demande de réexamen adressée à l'auteur de la décision administrative."),
    ("Recours contentieux", "Action en justice devant le tribunal administratif."),
    ("Cadastre", "Registre public définissant l'emplacement et la valeur des propriétés foncières."),
    ("Journal Officiel (JO)", "Publication officielle consignants lois, décrets et arrêtés."),
    ("Décret", "Acte réglementaire pris par le Président ou le Premier ministre."),
    ("Arrêté", "Décision administrative émanant d'un ministre, préfet ou maire."),
    ("Jurisprudence", "Ensemble des décisions de justice servant de référence."),

    # --- MACROÉCONOMIE & FINANCE ---
    ("PIB", "Indicateur mesurant la richesse produite sur le territoire national."),
    ("Inflation", "Hausse générale et durable des prix (perte pouvoir d'achat)."),
    ("Plus-value", "Gain réalisé lors de la vente d'un bien à un prix supérieur à l'achat."),
    ("OAT (Obligation Assimilable du Trésor)", "Titre d'emprunt d'État à moyen ou long terme."),
    ("Taux directeur", "Taux d'intérêt fixé par la Banque Centrale influençant le coût du crédit."),
    ("Déficit structurel", "Déficit budgétaire corrigé des variations conjoncturelles (indépendant de la croissance)."),
    ("Règle d'or budgétaire", "Principe visant l'équilibre des comptes publics sur le cycle économique."),
    ("Zone Euro", "Ensemble des pays de l'UE ayant adopté l'euro comme monnaie unique.")
]

# ==========================================
# 2. MEGA DATASET Q&A (50 Questions)
# ==========================================
dataset_qa = [
    # Usager Particulier
    ("Quel est l'impôt que je paie sur ce que je gagne chaque mois ?", "IR (Impôt sur le Revenu)"),
    ("J'ai acheté une baguette, j'ai payé une taxe sans le savoir, laquelle ?", "TVA"),
    ("Je viens d'hériter de ma tante, que réclame l'État ?", "Droits de succession"),
    ("Je suis propriétaire, je reçois un avis en octobre, c'est quoi ?", "Taxe Foncière"),
    ("Quelle taxe finance la sécu et apparait sur ma fiche de paie ?", "CSG"),
    ("Je vends mes actions en bourse, quel impôt forfaitaire s'applique ?", "PFU (Flat Tax)"),
    ("Je veux réduire mes impôts en embauchant une femme de ménage, c'est quel dispositif ?", "Niche fiscale"),
    ("J'ai 3 enfants, cela réduit mes impôts grâce à quel système ?", "Quotient familial"),
    ("J'ai payé trop d'impôts, le fisc me rembourse, ça s'appelle comment ?", "Dégrèvement"),
    ("On me saisit mon salaire directement à la banque pour impayé, c'est quoi ?", "Avis à tiers détenteur (ATD)"),
    ("Combien de temps le fisc peut-il remonter en arrière pour me contrôler ?", "Prescription fiscale"),
    ("Je ne suis pas d'accord avec le fisc, je demande à revoir la décision au directeur, c'est quel recours ?", "Recours gracieux"),

    # Entreprise / Pro
    ("C'est quoi le papier qui prouve que ma boite existe légalement ?", "Kbis"),
    ("Ma société a fait des bénéfices, je dois payer quoi ?", "IS (Impôt sur les Sociétés)"),
    ("Je dois déclarer mes échanges de services avec l'Allemagne, j'ai besoin de quel numéro ?", "TVA Intracommunautaire"),
    ("C'est quoi la différence entre ce que je possède et ce que je dois dans mes comptes ?", "Bilan comptable"),
    ("Je manque de cash pour payer mes fournisseurs avant d'être payé, c'est quel problème ?", "BFR (Besoin en Fonds de Roulement)"),
    ("Où je peux voir si j'ai fait du profit cette année ?", "Compte de résultat"),
    ("Je cherche l'identifiant à 14 chiffres de mon magasin.", "SIRET"),
    ("L'usure de mes machines fait baisser leur valeur, je note ça comment ?", "Amortissement comptable"),
    ("Je dois déposer mes factures pour la mairie sur quel site ?", "Chorus Pro"),
    ("C'est quoi la taxe sur les bureaux ou locaux de mon entreprise ?", "CFE"),
    ("Mon entreprise pollue, je paie quoi ?", "TGAP"),
    ("L'administration vient dans mes bureaux vérifier mes comptes, c'est quoi ?", "Vérification de comptabilité"),

    # Gestion Publique / LOLF / Admin
    ("L'État dépense plus qu'il ne gagne, c'est quelle situation ?", "Déficit budgétaire"),
    ("C'est quoi le montant total que la France doit aux banques ?", "Dette publique"),
    ("Qui a le droit de manipuler l'argent public ?", "Comptable public"),
    ("Qui décide d'acheter mais ne touche pas l'argent ?", "Ordonnateur"),
    ("C'est quoi la loi qui vote le budget pour l'an prochain ?", "Loi de Finances (PLF)"),
    ("Je dois déplacer des crédits d'une ligne à l'autre, j'utilise quel principe ?", "Fongibilité des crédits"),
    ("C'est quoi l'étape où on vérifie que le travail a bien été fait avant de payer ?", "Service fait"),
    ("L'ordre de payer donné au comptable, ça s'appelle comment ?", "Mandatement"),
    ("Le comptable ne peut pas recouvrer une dette car le débiteur est insolvable, il fait quoi ?", "Admission en non-valeur"),
    ("C'est quoi le document technique précis pour un marché public ?", "CCTP"),
    ("Je cherche les limites de dépenses juridiques que je peux signer.", "AE (Autorisations d'Engagement)"),
    ("C'est quoi la limite de cash que je peux sortir cette année ?", "CP (Crédits de Paiement)"),

    # Économie / Droit
    ("Les prix augmentent au supermarché, c'est quel indicateur ?", "Inflation"),
    ("La richesse créée par la France en 2024, c'est quel chiffre ?", "PIB"),
    ("J'ai revendu mon terrain plus cher, j'ai réalisé une...", "Plus-value"),
    ("La Banque Centrale change le coût de l'argent, elle touche à quoi ?", "Taux directeur"),
    ("Où sont publiées les nouvelles lois chaque matin ?", "Journal Officiel (JO)"),
    ("C'est quoi une décision prise par le Premier Ministre ?", "Décret"),
    ("L'ensemble des décisions des juges qui servent d'exemple, c'est...", "Jurisprudence"),
    ("Je veux savoir les limites de ma parcelle de terrain.", "Cadastre"),
    ("L'État emprunte de l'argent sur 10 ans, il émet quoi ?", "OAT (Obligation Assimilable du Trésor)"),
    ("C'est quoi la taxe spécifique sur le rhum en Guadeloupe ?", "Octroi de mer"),
    ("C'est quoi la taxe sur le tabac ou l'essence ?", "Accises"),
    ("Une entreprise crée un montage artificiel pour ne pas payer, c'est quoi ?", "Abus de droit"),
    ("C'est quoi la différence entre fraude et optimisation ?", "Fraude fiscale"), # Question piège, on attend qu'il relie à Fraude s'il y a illégalité
    ("L'Europe partage une monnaie, c'est quelle zone ?", "Zone Euro")
]

# ==========================================
# 3. MEGA DATASET TRIPLETS (50 Pièges Logiques)
# ==========================================
# Format : (ANCRE, POSITIF, NÉGATIF/PIÈGE)
dataset_triplets = [
    ("TVA", "Consommation", "Revenu"),
    ("IS (Impôt Sociétés)", "Bénéfice", "Chiffre d'affaires"),
    ("Bilan comptable", "Patrimoine", "Rentabilité"),
    ("Compte de résultat", "Flux", "Stock"),
    ("Amortissement", "Perte de valeur", "Décaissement d'argent"), # Amortissement = charge non décaissée
    ("Provision", "Risque", "Dette certaine"),
    ("Kbis", "Identité", "Bilan"),
    ("SIRET", "Établissement", "Entreprise"), # SIREN = Entreprise, SIRET = Établissement
    ("Recouvrement", "Encaisser", "Dépenser"),
    ("Ordonnateur", "Décideur", "Payeur"), # Séparation des pouvoirs
    ("Comptable public", "Payeur", "Décideur"),
    ("Mandatement", "Ordre de payer", "Paiement effectif"),
    ("Service fait", "Réalité", "Commande"),
    ("Loi de Finances", "Budget prévisionnel", "Bilan passé"), # PLF vs Loi de Règlement
    ("Loi de Règlement", "Constat", "Prévision"),
    ("Déficit", "Flux annuel", "Stock cumulé"), # Déficit (an) vs Dette (stock)
    ("Dette publique", "Stock", "Flux"),
    ("AE (Autorisations d'Engagement)", "Juridique", "Trésorerie"),
    ("CP (Crédits de Paiement)", "Trésorerie", "Juridique"),
    ("Fongibilité", "Souplesse", "Rigidité"),
    ("Marché public", "Contrat", "Subvention"), # Marché = contrepartie, Subvention = sans contrepartie directe
    ("Appel d'offres", "Concurrence", "Gré à gré"),
    ("CCTP", "Technique", "Administratif"),
    ("CCAG", "Général", "Particulier"),
    ("Redevance", "Service rendu", "Impôt"), # Distinction clé administrative
    ("Taxe", "Affectée", "Universelle"),
    ("Impôt", "Sans contrepartie", "Service rendu"),
    ("CSG", "Social", "Fiscal"), # Hybride mais finance le social
    ("Taxe Foncière", "Propriétaire", "Locataire"),
    ("Taxe d'habitation", "Occupant", "Propriétaire"),
    ("Niche fiscale", "Dérogation", "Droit commun"),
    ("Abus de droit", "Fictif", "Légal"),
    ("Fraude fiscale", "Illégal", "Optimisation"),
    ("Rescrit fiscal", "Garantie", "Contrôle"),
    ("ATD", "Saisie", "Avis amiable"),
    ("Prescription", "Fin de délai", "Poursuite"),
    ("Cadastre", "Foncier", "Commercial"),
    ("Inflation", "Prix", "Croissance"),
    ("PIB", "Production", "Patrimoine"),
    ("OAT", "Emprunt", "Action"),
    ("Taux directeur", "Banque Centrale", "Banque commerciale"),
    ("Décret", "Exécutif", "Législatif"), # Pris par gvt, pas voté par parlement
    ("Loi", "Parlement", "Gouvernement"),
    ("Jurisprudence", "Juge", "Député"),
    ("Arrêté", "Local/Ministériel", "National/Légal"),
    ("Chorus Pro", "Dématérialisation", "Papier"),
    ("Titre de recette", "Recette", "Dépense"),
    ("Reste à recouvrer", "Impayé", "Encaissé"),
    ("Admission en non-valeur", "Abandon", "Report")
]

# ==========================================
# 4. CONFIGURATION ET LANCEMENT
# ==========================================

# Liste des modèles (Configuration optimisée pour Colab/Standard)
models_config = [
    {"name": "Solon (Juridique FR)", "id": "OrdalieTech/Solon-embeddings-large-0.1"},
    {"name": "Snowflake Arctic v2.0", "id": "Snowflake/snowflake-arctic-embed-m-v2.0"},
    {"name": "BGE-M3 (Multilingue)", "id": "BAAI/bge-m3"},
    {"name": "GTE-Multilingual", "id": "Alibaba-NLP/gte-multilingual-base"},
    {"name": "E5-Large Instruct", "id": "intfloat/multilingual-e5-large-instruct"}
]

# Préparation des listes plates pour l'encodage
termes_bercy = [item[0] for item in dataset_bercy]
defs_bercy = [item[1] for item in dataset_bercy]

def run_benchmark(model, model_name):
    # Gestion des préfixes pour E5
    prefix_q = "query: " if "e5" in model_name.lower() or "instruct" in model_name.lower() else ""
    prefix_d = "passage: " if "e5" in model_name.lower() or "instruct" in model_name.lower() else ""

    print(f"   🔹 Encodage ({len(termes_bercy)} termes)...")

    # 1. DEFINITIONS (Top-1 Accuracy)
    inputs_termes = [prefix_q + t for t in termes_bercy]
    inputs_defs = [prefix_d + d for d in defs_bercy]

    emb_termes = model.encode(inputs_termes, convert_to_tensor=True, show_progress_bar=False)
    emb_defs = model.encode(inputs_defs, convert_to_tensor=True, show_progress_bar=False)

    scores = util.cos_sim(emb_termes, emb_defs)
    top1 = 0
    for i in range(len(termes_bercy)):
        if torch.argmax(scores[i]).item() == i:
            top1 += 1
    score_defs = (top1 / len(termes_bercy)) * 100

    # 2. Q&A (Retrieval Accuracy)
    q_inputs = [prefix_q + q[0] for q in dataset_qa]
    emb_q = model.encode(q_inputs, convert_to_tensor=True, show_progress_bar=False)

    qa_hits = 0
    # On compare les Questions (emb_q) aux Termes (emb_termes déjà calculé)
    scores_qa = util.cos_sim(emb_q, emb_termes)

    for i, item in enumerate(dataset_qa):
        target = item[1]
        try:
            target_idx = termes_bercy.index(target)
            if torch.argmax(scores_qa[i]).item() == target_idx:
                qa_hits += 1
        except ValueError: continue
    score_qa = (qa_hits / len(dataset_qa)) * 100

    # 3. TRIPLETS (Logic Score)
    ancres = [prefix_q + t[0] for t in dataset_triplets]
    positifs = [prefix_d + t[1] for t in dataset_triplets]
    negatifs = [prefix_d + t[2] for t in dataset_triplets]

    emb_a = model.encode(ancres, convert_to_tensor=True, show_progress_bar=False)
    emb_p = model.encode(positifs, convert_to_tensor=True, show_progress_bar=False)
    emb_n = model.encode(negatifs, convert_to_tensor=True, show_progress_bar=False)

    sim_ap = torch.nn.functional.cosine_similarity(emb_a, emb_p)
    sim_an = torch.nn.functional.cosine_similarity(emb_a, emb_n)

    triplet_succ = 0
    for i in range(len(dataset_triplets)):
        if sim_ap[i] > sim_an[i]: triplet_succ += 1
    score_triplets = (triplet_succ / len(dataset_triplets)) * 100

    return {
        "Modèle": model_name,
        "Déf. (Top-1)": score_defs,
        "Q&A (User)": score_qa,
        "Triplets (Logic)": score_triplets,
        "Moyenne": (score_defs + score_qa + score_triplets) / 3
    }

# BOUCLE PRINCIPALE
results = []
print(f"🚀 LANCEMENT DU MEGA-BENCHMARK ({len(models_config)} modèles sur {len(termes_bercy)} termes)")

for cfg in models_config:
    print(f"\n📥 Test de : {cfg['name']}")
    try:
        model = SentenceTransformer(cfg['id'], trust_remote_code=True)
        res = run_benchmark(model, cfg['name'])
        results.append(res)
        print(f"   ✅ Score Global : {res['Moyenne']:.1f}%")

        del model
        gc.collect()
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"   ❌ Erreur : {e}")

# TABLEAU FINAL
df = pd.DataFrame(results).sort_values(by="Moyenne", ascending=False)
print("\n🏆 CLASSEMENT FINAL : MEGA BERCY 🏆")
print(df.to_markdown(index=False, floatfmt=".1f"))

🚀 LANCEMENT DU MEGA-BENCHMARK (5 modèles sur 87 termes)

📥 Test de : Solon (Juridique FR)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

   🔹 Encodage (87 termes)...
   ✅ Score Global : 39.3%

🏆 CLASSEMENT FINAL : MEGA BERCY 🏆
| Modèle               |   Déf. (Top-1) |   Q&A (User) |   Triplets (Logic) |   Moyenne |
|:---------------------|---------------:|-------------:|-------------------:|----------:|
| E5-Large Instruct    |           39.1 |         32.0 |               46.9 |      39.3 |
| Solon (Juridique FR) |           35.6 |         20.0 |               53.1 |      36.2 |
| BGE-M3 (Multilingue) |           27.6 |         18.0 |               59.2 |      34.9 |
| GTE-Multilingual     |           34.5 |         24.0 |               44.9 |      34.5 |
